# Myxococcales OTUs and rdp type strain isolate tree
* Align all Myxococcales OTUs from sequencing run with type strains from RDP 
* Desulfobacteria used as outgroup


In [11]:
workDir = '/home/be68/Hyphosphere/data/Myxococcales/OTUtree/'
#OTUFileDir = '/home/be68/Hyphosphere/data/Myxococcales/'
RDPfile = 'rdp_download_51seqs.fa'
OTUFile = '/home/be68/Hyphosphere/data/Myxococcales/OTUtree/MyxOTUseqs.fa'


nprocs = 3

In [2]:
import os, re
import numpy as np
#import entrez.direct
from cogent.app.fasttree import build_tree_from_alignment
from cogent import DNA, LoadSeqs
from Bio import Entrez
Entrez.email = "be68@cornell.edu"

In [3]:
if not os.path.isdir(workDir):
    os.mkdir(workDir)

In [4]:
os.chdir(workDir)

In [11]:
# sso_acc = "KJ475674"
# sso_fa = Entrez.efetch(db="nucleotide", id=sso_acc, rettype="fasta", retmode="text").readlines()

In [14]:
%%bash -s "$OTUFile" "$RDPfile" 
#head $1 -n 15
head $1 -n 15
head $2 -n 8


>Otu10
TACGAAGGGTGCAAACGTTGCTCGGAATCATTGGGCGTAAAGCGCACGTAGGCGGCTTTGCAAGTCGGATGTGAAATCCCGTGGCTCAACCATGGACGTGCATTCGAAACTGCAAAGCTTGAGTATGGAAGAGGGTCGCGGAATTCCCGGTGTAGAGGTGAAATTCGTAGATATCGGGAGGAACACCAGTGGCGAAGGCGGCGACCTGGGCCAATACTGACGCTGAGGTGCGAAAGCGTGGGGAGCAAACAGG
>Otu52
GACAGAGGGTGCAAACGTTGTTCGGAATTACTGGGCGTAAAGCGCGTGTAGGCGGCCCTGCAAGTCGGATGTGAAATCCCACGGCTCAACCATGGAAGTGCATTCGAAACTGCAGAGCTAGAGTCCTGGAGAGGAAGGTGGAATGCTTGGTGTAGAGGTGAAATTCGTAGATATCAAGCGGAACACCGGTGGCGAAGGCGGCCTTCTGGACAGTGACTGACGCTGAGACGCGAAAGCGTGGGGAGCAAACAGG
>Otu86
GACAGAGGGTGCAAACGTTGTTCGGAATTACTGGGCGTAAAGCGCGTGTAGGCGGTTCGTAAAGTCAGGTGTGAAAGCCCTGGGCTTAACCCAGGAAGTGCACTTGAAACTCACGAACTTGAGTCCCGGAGAGGAAGGCGGAATTCTCGGTGTAGAGGTGAAATTCGTAGATATCGAGAGGAACATCGGTGGCGAAGGCGGCCTTCTGGACGGTGACTGACGCTGAGACGCGAAAGCGTGGGGAGCAAACAGG
>Otu97
GACAGAGGGTGCGAACGTTGTTCGGAATTACTGGGCGTAAAGCGCGTGTAGGCGGCAGCGTGTGTCGGGCGTGAAATCCCATGGCTTAACCATGGAAGTGCACCCGAAACTGCGCAGCTAGAGTCCTGGAGAGGAAGATGGAATACTTGGTGTAGAGGTGAAATTCGTAGATATCAAGTGGAACACCGGTGGCGAAGGCGGTCTTCTGGA

In [24]:
## attempt to remove gaps in python, easier to do in nano.
# handle = open("SeqFile.fa")
# #counter = 0
# for line in handle:
#     if line.startswith(">"):
#         #counter = counter + 1
#         continue
#     line = re.sub('-','',line)   
#     print(line)
        

## Using SSU-Align to seqs from hyphal samples and mask based on alignment posterior probabilities.

### Align hyphal OTUs

In [17]:
%%bash -s "$OTUFile"
ssu-prep -f -x -b 50 --rfonly --dna $1 ssu_aln 2

# _ssu-prep :: prepare SSU rRNA sequences for parallel ssu-align jobs
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-prep -x -f -b 50 --dna --rfonly /home/be68/Hyphosphere/data/Myxococcales/OTUtree/MyxOTUseqs.fa ssu_aln 2
# date:    Mon Feb 26 12:19:08 2018
#
# Validating input sequence file ... done.
#
# Preparing 2 ssu-align jobs ...
# Partitioning seqs with goal of equalizing total number of nucleotides per job ...
#
# output file name      description                                        
# --------------------  ---------------------------------------------------
  ssu_aln/MyxOTUseqs.fa.1  partition 1 FASTA sequence file (48 seqs; 12144 nt)
  ssu_aln/MyxOTUseqs.fa.2  partition 2 FASTA sequence file (48 seqs; 12144 nt)
  ssu_aln.ssu-align.sh  shell script that will execute 2 ssu-align jobs
#
###########

In [18]:
%%bash
./ssu_aln.ssu-align.sh

# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/MyxOTUseqs.fa.1 ssu_aln/ssu_aln.1 > /dev/null &
# Executing: ssu-align --merge 2 -b 50 --dna --rfonly ssu_aln/MyxOTUseqs.fa.2 ssu_aln/ssu_aln.2
# _ssu-align :: align SSU rRNA sequences
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-align --merge 2 -b 50 --dna --rfonly ssu_aln/MyxOTUseqs.fa.2 ssu_aln/ssu_aln.2
# date:    Mon Feb 26 12:19:15 2018
#
# Validating input sequence file ... done.
#
# Stage 1: Determining SSU start/end positions and best-matching models...
#
# output file name            description                                
# --------------------------  -------------------------------------------
  ssu_aln.2.tab               locations/scores of hits defined by HMM(s)
  ssu_aln.2.bacteria.hitlist  list of sequences to align with bacteria CM
 

In [19]:
!cd $workDir; ssu-mask --dna --afa ssu_aln

# _ssu-mask :: mask SSU rRNA alignments
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-mask --afa --dna ssu_aln
# date:    Mon Feb 26 12:19:27 2018
#
# Masking alignments based on posterior probabilities...
#
#                                                     mask    
#                                                 ------------
# file name                  in/out  type  #cols  incl.  excl.
# -------------------------  ------  ----  -----  -----  -----
  ssu_aln.bacteria.stk        input   aln   1582      -      -
  ssu_aln.bacteria.mask      output  mask   1582    251   1331
  ssu_aln.bacteria.mask.pdf  output   pdf   1582    251   1331
  ssu_aln.bacteria.mask.afa  output   aln    251      -      -
#
# All attempts to draw structure diagrams of masks were successful.
#
# List of executed commands saved 

### align RDP sequences

In [20]:
%%bash -s "$RDPfile"
ssu-prep -f -x -b 50 --rfonly --dna $1 ssr_aln 10

# _ssu-prep :: prepare SSU rRNA sequences for parallel ssu-align jobs
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-prep -x -f -b 50 --dna --rfonly rdp_download_51seqs.fa ssr_aln 10
# date:    Mon Feb 26 12:19:58 2018
#
# Validating input sequence file ... done.
#
# Preparing 10 ssu-align jobs ...
# Partitioning seqs with goal of equalizing total number of nucleotides per job ...
#
# output file name         description                                      
# -----------------------  -------------------------------------------------
  ssr_aln/rdp_download_51seqs.fa.1  partition 1 FASTA sequence file (6 seqs; 9165 nt)
  ssr_aln/rdp_download_51seqs.fa.2  partition 2 FASTA sequence file (6 seqs; 8944 nt)
  ssr_aln/rdp_download_51seqs.fa.3  partition 3 FASTA sequence file (5 seqs; 7520 nt)
  ssr_aln/rdp_downloa

In [21]:
%%bash
./ssr_aln.ssu-align.sh

# Executing: ssu-align -b 50 --dna --rfonly ssr_aln/rdp_download_51seqs.fa.1 ssr_aln/ssr_aln.1 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssr_aln/rdp_download_51seqs.fa.2 ssr_aln/ssr_aln.2 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssr_aln/rdp_download_51seqs.fa.3 ssr_aln/ssr_aln.3 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssr_aln/rdp_download_51seqs.fa.4 ssr_aln/ssr_aln.4 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssr_aln/rdp_download_51seqs.fa.5 ssr_aln/ssr_aln.5 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssr_aln/rdp_download_51seqs.fa.6 ssr_aln/ssr_aln.6 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssr_aln/rdp_download_51seqs.fa.7 ssr_aln/ssr_aln.7 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssr_aln/rdp_download_51seqs.fa.8 ssr_aln/ssr_aln.8 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssr_aln/rdp_download_51seqs.fa.9 ssr_aln/ssr_aln.9 > /dev/null &
# Executing: ssu-al

In [22]:
!cd $workDir; ssu-mask -s ssu_aln/ssu_aln.bacteria.mask --dna --afa ssr_aln/

# _ssu-mask :: mask SSU rRNA alignments
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-mask -s ssu_aln/ssu_aln.bacteria.mask --afa --dna ssr_aln/
# date:    Mon Feb 26 12:20:42 2018
#
# Masking alignments using pre-existing masks...
#
#                                                     mask    
#                                                 ------------
# file name                  in/out  type  #cols  incl.  excl.
# -------------------------  ------  ----  -----  -----  -----
  ssr_aln.bacteria.stk        input   aln   1582      -      -
  ssu_aln.bacteria.mask       input  mask   1582    251   1331
  ssr_aln.bacteria.mask.pdf  output   pdf   1582    251   1331
  ssr_aln.bacteria.mask.afa  output   aln    251      -      -
#
# All attempts to draw structure diagrams of masks were successful.
#
# List o

In [23]:
!cd $workDir; cat ssr_aln/ssr_aln.bacteria.mask.afa ssu_aln/ssu_aln.bacteria.mask.afa > aln_for_tree.fasta

In [24]:
!head aln_for_tree.fasta -n 20

>S002304630 Desulfobacter hydrogenophilus (T); type strain: DSM3380; FR733673
CACGGGGGGCGCAAGCGTTATTCGGAATTATTGGGCGTAAAGGGCGCGCAGGCGGTCTTG
TCCGTCAGGTGTGAAAGCTCGGGGCTCAACCCCGGAAGTGCACTTGAAACAGCAAGACTT
GAATACGGGAGAGGAGAGAGGAATTCCTGGTGTAGAGGTGAAATTCGTAGATATCAGGAG
GAACACCGATGGCGAAGGCATCTCTCTGGACCGATTGACGCTGAGGCGCGAAGGCGTGGG
TAGCGAACGGG
>S002287043 Anaeromyxobacter dehalogenans (T); 2CP1; CP001359
TACAGAGGGTGCGAGCGTTGTTCGGAATTATTGGGCGTAAAGCGCGTGTAGGCGGCCTAG
CAAGTCGGATGTGAAAGCCCTCGGCTTAACCGAGGAAGTGCGTTCGAAACTACTGGGCTT
GAGTACCGGAGAGGGTGGCGGAATTCCCGGTGTAGAGGTGAAATTCGTAGATATCGGGAG
GAACACCAGTGGCGAAGGCGGCCACCTGGACGGACTGACGCTGAGACGCGAAAGCGTGGG
TAGCAAACAGG
>S004068953 Anaeromyxobacter dehalogenans (T); 2CP1; CP001359
TACAGAGGGTGCGAGCGTTGTTCGGAATTATTGGGCGTAAAGCGCGTGTAGGCGGCCTAG
CAAGTCGGATGTGAAAGCCCTCGGCTTAACCGAGGAAGTGCGTTCGAAACTACTGGGCTT
GAGTACCGGAGAGGGTGGCGGAATTCCCGGTGTAGAGGTGAAATTCGTAGATATCGGGAG
GAACACCAGTGGCGAAGGCGGCCACCTGGACGGACTGACGCTGAGACGCGAAAGCGTGGG
TAGCAAACAGG
>S000539285 Melittangium bolet

## Inferring and rooting the tree

In [25]:
aln = LoadSeqs(os.path.join(workDir, 'aln_for_tree.fasta'), moltype=DNA)
t_unroot = build_tree_from_alignment(aln, moltype=DNA)

In [26]:
outgroup = 'S002304630 Desulfobacter hydrogenophilus (T); type strain: DSM3380; FR733673'

In [27]:
t_rooted = t_unroot.rootedWithTip(outgroup)

In [28]:
t_rooted.writeToFile(os.path.join(workDir, 'MyxOTUs_wTypeStrains.tree'))